<a href="https://colab.research.google.com/github/RohitPatilRRP/valueLabs/blob/master/valueLabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
import nltk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [180]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [181]:
url_train = 'https://raw.githubusercontent.com/RohitPatilRRP/valueLabs/master/Train.csv'
train = pd.read_csv(url_train)
train.columns = [x.lower() for x in train.columns]
train.head()

,question,answer_text,distractor
0,Meals can be served,in rooms at 9:00 p. m.,"'outside the room at 3:00 p. m.', 'in the dini..."
1,It can be inferred from the passage that,The local government can deal with the problem...,"'If some tragedies occur again ', ' relevant d..."
2,The author called Tommy 's parents in order to,help them realize their influence on Tommy,"'blame Tommy for his failing grades', 'blame T..."
3,It can be inferred from the passage that,the writer is not very willing to use idioms,'idioms are the most important part in a langu...
4,How can we deal with snake wounds according to...,Stay calm and do n't move .,'Cut the wound and suck the poison out .'


In [182]:
url_test = 'https://raw.githubusercontent.com/RohitPatilRRP/valueLabs/master/Test.csv'
test = pd.read_csv(url_test)
test.columns = [x.lower() for x in test.columns]
test.head()

,question,answer_text
0,What 'S the main idea of the text ?,The lack of career -- based courses in US high...
1,"In the summer high season , Finland does nt se...",the sun is out at night
2,If you want to apply for Chinese Business Inte...,have to get confirmed at least twice
3,"That afternoon , the boy 's clothes were dry b...",nobody made room for him in the water .
4,Which of the following statements is NOT true ?,There are twelve countries in the World Wildli...


In [0]:
x_train = train.iloc[:,0:2]
y_train  = train.iloc[:,2:3]
x_test = train.iloc[:,0:2] 
y_test  = train.iloc[:,2:3]

In [0]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import keras.utils as ku 
from nltk.tokenize import word_tokenize , sent_tokenize

In [0]:
tokenizer_obj = Tokenizer()
total_qs = x_train['question'] + x_test['question']
total_ans = x_train['answer_text']+x_test['answer_text']

In [0]:
tokenizer_obj.fit_on_texts(total_qs)
tokenizer_obj.fit_on_texts(total_ans)

In [199]:
maxlen_qs = max([len(s.split()) for s in total_qs])
maxlen_qs

105

In [200]:
maxlen_ans = max([len(s.split()) for s in total_ans])
maxlen_ans

229

In [0]:
vocab_size = len(tokenizer_obj.word_index) + 1

In [0]:
x_train_qs_token = tokenizer_obj.texts_to_sequences(x_train['question'])
x_train_ans_token = tokenizer_obj.texts_to_sequences(x_train['answer_text'])
x_test_qs_token = tokenizer_obj.texts_to_sequences(x_train['question'])
x_test_ans_token = tokenizer_obj.texts_to_sequences(x_train['answer_text'])

In [0]:
x_train_qs_pad = pad_sequences(x_train_qs_token,maxlen=maxlen_ans,padding='post')
x_train_ans_pad = pad_sequences(x_train_ans_token,maxlen=maxlen_ans,padding='post')
x_test_qs_pad = pad_sequences(x_test_qs_token,maxlen=maxlen_ans,padding='post')
x_test_ans_pad = pad_sequences(x_test_ans_token,maxlen=maxlen_ans,padding='post')

In [232]:
total_ds = y_train+y_test
len(total_ds)

31499

In [0]:
# tokenizer_obj.fit_on_texts(total_ds)
# maxlen_ds = max([len(s.split()) for s in total_ds])
# y_train_token = tokenizer_obj.texts_to_sequences(y_train)
# y_test_token = tokenizer_obj.texts_to_sequences(y_test)

# y_train_pad = pad_sequences(y_train_token,maxlen=maxlen_ds,padding='post')
# y_test_pad = pad_sequences(y_test_token,maxlen=maxlen_ds,padding='post')

In [228]:
len(y_train_pad)

1

In [221]:
from keras.models import Sequential
from keras.layers import Dense ,Embedding ,LSTM ,GRU
from keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100
print("Building Model..")
model = Sequential()
model.add(Embedding(vocab_size,EMBEDDING_DIM,input_length=maxlen_ans))
model.add(GRU(units=32 ,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='relu'))

model.compile(loss="binary_crossentropy", optimizer='adam',
	metrics=["accuracy"])

Building Model..


In [0]:
final_x_test = x_test_qs_pad + x_test_ans_pad

In [216]:
final_x= x_train_qs_pad + x_train_ans_pad
final_x

array([[ 1262,  2382,    66, ...,     0,     0,     0],
       [   28,   565,   355, ...,     0,     0,     0],
       [   81,   156,  1287, ...,     0,     0,     0],
       ...,
       [  209,   426,   239, ...,     0,     0,     0],
       [37818,    68,  8049, ...,     0,     0,     0],
       [   35,   388,  5145, ...,     0,     0,     0]], dtype=int32)

In [233]:
model.fit(final_x,y_train,batch_size=128,epochs=25,validation_data=(final_x_test,y_test_pad),verbose=2)

ValueError: ignored